In [20]:
from pymatgen.io.lammps.data import *
from pymatgen.core.structure import *

In [2]:
system = LammpsData.from_file("data.oleic_xligand.test")
xlig_dat = LammpsData.from_file("data.xligand")
oleic_dat = LammpsData.from_file("data.oleic_acid")

xlig = xlig_dat.structure
oleic = oleic_dat.structure

box, ff, tops = system.disassemble()
forcefield = ff.as_dict()

In [33]:
tops[0].as_dict()

{'@module': 'pymatgen.io.lammps.data',
 '@class': 'Topology',
 '@version': None,
 'sites': {'@module': 'pymatgen.core.structure',
  '@class': 'Molecule',
  'charge': 0.0,
  'spin_multiplicity': 1,
  'sites': [{'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [23.52343, 33.65611, 53.99145],
    'properties': {'ff_map': 'C1'}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [23.15767, 32.34818, 53.26003],
    'properties': {'ff_map': 'C1'}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [24.4817, 31.85062, 52.64144],
    'properties': {'ff_map': 'C1'}},
   {'name': 'N',
    'species': [{'element': 'N', 'occu': 1}],
    'xyz': [25.54171, 32.46851, 53.47921],
    'properties': {'ff_map': 'N1'}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [26.75328, 32.51001, 52.79901],
    'properties': {'ff_map': 'C2'}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [27.64

In [18]:
len(system.structure[68:])

54

In [23]:
system.structure[68:]

[PeriodicSite: O (51.6913, 31.1431, 9.8705) [0.8615, 0.5191, 0.1645],
 PeriodicSite: O (53.0083, 29.3098, 9.6294) [0.8835, 0.4885, 0.1605],
 PeriodicSite: C (57.0337, 31.4450, 4.6484) [0.9506, 0.5241, 0.0775],
 PeriodicSite: C (58.2032, 30.7210, 3.9720) [0.9701, 0.5120, 0.0662],
 PeriodicSite: C (56.3137, 30.5881, 5.6926) [0.9386, 0.5098, 0.0949],
 PeriodicSite: C (57.8327, 29.4873, 3.1425) [0.9639, 0.4915, 0.0524],
 PeriodicSite: C (55.1816, 31.3300, 6.4111) [0.9197, 0.5222, 0.1069],
 PeriodicSite: C (52.3007, 29.0327, 3.4090) [0.8717, 0.4839, 0.0568],
 PeriodicSite: C (51.0874, 29.8373, 2.9361) [0.8515, 0.4973, 0.0489],
 PeriodicSite: C (53.2664, 28.7555, 2.2511) [0.8878, 0.4793, 0.0375],
 PeriodicSite: C (56.8281, 29.7827, 2.0261) [0.9471, 0.4964, 0.0338],
 PeriodicSite: C (54.5123, 30.4949, 7.5083) [0.9085, 0.5082, 0.1251],
 PeriodicSite: C (50.0538, 30.0835, 4.0392) [0.8342, 0.5014, 0.0673],
 PeriodicSite: C (54.5134, 27.9771, 2.6839) [0.9086, 0.4663, 0.0447],
 PeriodicSite: C (56

In [22]:
test = IMolecule(system.structure[68:])

TypeError: IMolecule.__init__() missing 1 required positional argument: 'coords'

In [12]:
len(oleic) + len(xlig)

122

In [6]:
def get_ff_type(idx,structure):
    site = structure.as_dict()['sites']['sites'][idx] # preserves original index of atoms from lammps data file
    ff_type = site['properties']['ff_map']  # pull the forcefield type from the structure-topo dict
    return ff_type

def get_bond_type(bond,ff_dict):
    
    bond_type = ""
    
    for i in range(len(ff_dict['topo_coeffs']["Bond Coeffs"])):
        # print(bond, ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]) 
        if bond == ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]:
            bond_type = i
        elif bond[::-1] == ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]:
            bond_type = i
    
    if type(bond_type) == int:
        return bond_type
    else:
        raise Exception("type not found")
        
# this function expects:
# -- relation should be an array of atom types as imported by pymatgen from lammps datafile 
# -- eg [C1, H1, C1, N1]
# -- ff_dict should be the output of the ff.as_dict() pymatgen lammps forcefield object 
# -- topology should be the name of the relation you want to type (Bonds, Angles, Dihedrals, Impropers)
def get_topo_type(relation, ff_dict, topology):
    
    topo_type = ""
    topo_coeff = ""
    
    # convert standard topology name to singular + " Coeff" format for pymatgen ff_dict format
    topo = {"Bond", "Angle", "Dihedral", "Improper"}
    for i in topo:
        if i in topology:
            topo_coeff = i + " Coeffs"
    
    if topo_coeff == "":
        raise Exception("Topology not found")
    
    # check the original or reverse order of the topological relation (bond, angle...) 
    for i in range(len(ff_dict['topo_coeffs'][topo_coeff])):
        # print(relation, ff_dict['topo_coeffs'][topo_coeff][i]['types'][0])
        if relation in ff_dict['topo_coeffs'][topo_coeff][i]['types']:
            topo_type = i
        elif relation[::-1] in ff_dict['topo_coeffs'][topo_coeff][i]['types']:
            topo_type = i
    
    if type(topo_type) == int:
        return topo_type
    else:
        raise Exception("type not found")

# writes out the desired lammps topology in molecule-file compatible format 
# note the "+ 1"s to convert to lammps 1-based indexing
def get_lammps_topology(structure, ff_dict, topology):
    i = 0
    dat = []
    for neighbor in structure.topologies[topology]:
        i  += 1
        neighbor_types = [get_ff_type(x,structure) for x in neighbor] # get ff types for each element in the connected relation
        relation_type = get_topo_type(neighbor_types, ff_dict, topology) # get type of the relation
        lammps_bond = [str(i + 1) for i in neighbor] # convert to LAMMPS 1-based indexing
        # print(i, relation_type + 1, " ".join(lammps_bond))
        dat.append(f"{i} {relation_type + 1} {' '.join(lammps_bond)}\n")
    
    return dat

def write_lammps_topology(filename, topo_dat):
    with open(filename, "w") as f:
        for line in topo_dat:
            f.write(line)
            
def generate_neighbors(topology,structure):
    neighbors = {atom: set() for atom in range(1,len(structure)+1)}
    for relation in topology:
        neighbors[relation[0] + 1].add(relation[-1] + 1)
        neighbors[relation[-1] + 1].add(relation[0] + 1)
    return neighbors

def get_special_bonds(neighbors,structure):
    special_bonds = {atom: set() for atom in range(1, len(structure) + 1)}
    for top in ['Bonds', 'Angles', 'Dihedrals', 'Impropers']:
        for atom in special_bonds:
            special_bonds[atom] = special_bonds[atom].union(neighbors[top][atom])
    
    return special_bonds


def get_special_bond_counts(neighbors, structure):
    special_bond_counts = {atom: [] for atom in range(1, len(structure) + 1)}
    for top in ['Bonds', 'Angles', 'Dihedrals']:
        for atom in special_bond_counts:
            if top == "Bonds" or top == "Angles":
                special_bond_counts[atom].append(len(neighbors[top][atom]))
            elif top == "Dihedrals":
                count = 0
                for i in neighbors[top][atom]:
                    if i not in neighbors["Angles"][atom]:
                        count += 1
                special_bond_counts[atom].append(count)
    
    return special_bond_counts

In [7]:
topologies = ['Bonds', 'Angles', 'Dihedrals', 'Impropers']
neighbors = {topology: generate_neighbors(tops[1].topologies[topology], oleic) for topology in topologies}
special_bonds = get_special_bonds(neighbors,oleic)
special_bond_counts = get_special_bond_counts(neighbors,oleic)

In [43]:
coords = []
types = []
charges = []
with open("oleic_test.txt") as f:
    dat = f.readlines()
    
for i,line in enumerate(dat):
    raw = line.split()
    x = round(float(raw[4]),1)
    y = round(float(raw[5]),1)
    z = round(float(raw[6]),1)
    # pos = [int(raw[0]), x,y,z]
    # at = [int(raw[0]), int(raw[2])]
    # charge = [int(raw[0]), float(raw[3])]
    pos = [i+1, x,y,z]
    at = [i+1, int(raw[2])]
    charge = [i+1, float(raw[3])]
    charges.append(charge)
    types.append(at)
    coords.append(pos)

bonds = get_lammps_topology(tops[1], forcefield, 'Bonds')
angles = get_lammps_topology(tops[1], forcefield, 'Angles')
dihedrals = get_lammps_topology(tops[1], forcefield, 'Dihedrals')
impropers = get_lammps_topology(tops[1], forcefield, 'Impropers')

In [44]:
coords

[[1, 51.7, 31.1, 9.9],
 [2, 53.0, 29.3, 9.6],
 [3, 57.0, 31.4, 4.6],
 [4, 58.2, 30.7, 4.0],
 [5, 56.3, 30.6, 5.7],
 [6, 57.8, 29.5, 3.1],
 [7, 55.2, 31.3, 6.4],
 [8, 52.3, 29.0, 3.4],
 [9, 51.1, 29.8, 2.9],
 [10, 53.3, 28.8, 2.3],
 [11, 56.8, 29.8, 2.0],
 [12, 54.5, 30.5, 7.5],
 [13, 50.1, 30.1, 4.0],
 [14, 54.5, 28.0, 2.7],
 [15, 56.5, 28.5, 1.3],
 [16, 50.6, 30.9, 5.2],
 [17, 55.5, 27.7, 1.6],
 [18, 53.4, 31.2, 8.2],
 [19, 49.5, 31.1, 6.3],
 [20, 52.7, 30.4, 9.3],
 [21, 56.3, 31.8, 3.9],
 [22, 57.4, 32.3, 5.1],
 [23, 58.7, 31.4, 3.3],
 [24, 58.9, 30.4, 4.7],
 [25, 57.0, 30.2, 6.4],
 [26, 55.9, 29.7, 5.2],
 [27, 57.4, 28.7, 3.8],
 [28, 58.8, 29.1, 2.7],
 [29, 55.6, 32.3, 6.9],
 [30, 54.4, 31.6, 5.7],
 [31, 52.0, 28.1, 3.8],
 [32, 52.8, 29.6, 4.2],
 [33, 50.6, 29.3, 2.1],
 [34, 51.4, 30.8, 2.5],
 [35, 52.7, 28.2, 1.5],
 [36, 53.6, 29.7, 1.8],
 [37, 57.3, 30.5, 1.3],
 [38, 55.9, 30.2, 2.4],
 [39, 54.1, 29.6, 7.1],
 [40, 55.3, 30.2, 8.3],
 [41, 49.7, 29.1, 4.4],
 [42, 49.2, 30.6, 3.6],
 

In [45]:
with open("oleic.lammps.corrected.txt", "w") as f:
    
    f.write("Coords\n\n")
    for pos in coords:
        f.write(f"{pos[0]}    {pos[1]} {pos[2]} {pos[3]}\n")
        
    f.write("\nTypes\n\n")
    for at in types:
        f.write(f"{at[0]}    {at[1]}\n")
        
    f.write("\nCharges\n\n")
    for charge in charges:
        f.write(f"{charge[0]}    {charge[1]}\n")
    
    f.write("\nBonds\n\n")
    for bond in bonds:
        f.write(bond)
    
    f.write("\nAngles\n\n")
    for angle in angles:
        f.write(angle)
    
    f.write("\nDihedrals\n\n")
    for dihed in dihedrals:
        f.write(dihed)
        
    f.write("\nImpropers\n\n")
    for imp in impropers:
        f.write(imp)
    
    f.write("\nSpecial Bond Counts\n\n")
    for atom in special_bond_counts:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bond_counts[atom]])}\n")
        
    f.write("\nSpecial Bonds\n\n")
    for atom in special_bonds:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bonds[atom]])}\n")